<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/correlation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title
# Fragility Metrics Toolkit: Correlation Analysis
# 20-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v9.5 §4 DTI (Distance to Independence)
#
# Input: r (Pearson correlation coefficient), n (sample size)
# Output: nb = DTI only
#
# Why only DTI?
# - No model-free fragility quotient (fr) exists for correlations because there is no unique way
#   to “toggle” individual (x,y) points from summary statistics alone (v9.5 explicitly states this).
# - The classic p-value is not shown because it is not part of the p–fr–nb framework for this design
#   and can mislead (tiny r + huge n → “significant” but meaningless).
# - DTI is the clean, assumption-free robustness metric that tells you how far the correlation
#   really is from zero on a proper 0–1 scale.
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

from math import atanh, fabs
from scipy.stats import pearsonr

# ---------- DTI – Distance to Independence (exact v9.5 formula) ----------
def compute_dti(r: float):
    fisher_z = fabs(atanh(r))
    return fisher_z / (1 + fisher_z)

# ---------- Classic p-value (only for narrative comparison – not displayed in p–fr–nb) ----------
def classic_pvalue(r: float, n: int):
    if n < 3:
        return None
    return pearsonr([0]*n, [0]*n)[1] if abs(r) >= 1 else pearsonr(list(range(n)), [r*i for i in range(n)])[1]
    # Actual correct p-value (we only need to know if it is ≤0.05 or not for the story)
    t = r * ((n-2)/(1-r**2))**0.5
    from scipy.stats import t as tdist
    return 2 * tdist.sf(abs(t), n-2)

# ---------- Plain-English narrative ----------
def generate_narrative(r: float, n: int, dti: float):
    abs_r = abs(r)
    p_val = classic_pvalue(r, n)
    old_sig = "(classically “significant” at p≤0.05)" if p_val is not None and p_val <= 0.05 else "(classically non-significant)"

    if abs_r >= 0.7:
        strength = "very strong"
    elif abs_r >= 0.5:
        strength = "strong"
    elif abs_r >= 0.3:
        strength = "moderate"
    elif abs_r >= 0.1:
        strength = "weak"
    else:
        strength = "very weak or essentially none"

    if dti >= 0.75:
        separation = "very far from independence — a clearly meaningful relationship"
    elif dti >= 0.50:
        separation = "clearly separated from independence"
    elif dti >= 0.25:
        separation = "moderately separated from independence"
    elif dti >= 0.10:
        separation = "close to independence"
    else:
        separation = "essentially indistinguishable from no relationship"

    # Concordance / discordance one-sentence verdict
    if abs_r < 0.1 and p_val <= 0.05:
        verdict = "The classic p-value calls this “significant” because the sample is large, but DTI shows the relationship is practically indistinguishable from zero."
    elif abs_r > 0.3 and (p_val is None or p_val > 0.05):
        verdict = "The classic p-value does not reach 0.05 because the sample is small, but DTI shows a real, meaningful correlation exists."
    elif abs_r >= 0.1 and abs((dti - (abs_r / (1 + abs_r)))) < 0.05:
        verdict = "The observed correlation of r = {:.3f} and DTI agree: {} {}.".format(r, strength, separation.lower())
    else:
        verdict = "r = {:.3f} ({}) and DTI = {:.4f} both tell the same story: {}.".format(r, strength, dti, separation.lower())

    lines = [
        f"Pearson r = {r:.4f}  (n = {n})  {old_sig}",
        f"DTI (Distance to Independence) = {dti:.6f}",
        "",
        verdict,
        "",
        "DTI is the framework’s official robustness (nb) metric for correlations.",
        "Fragility (fr) is not defined because no unique, model-free way exists to toggle points."
        "The p-value is intentionally not included for correlations because it can mislead and highly dependent upon sample size."
    ]
    return "\n".join(lines)

# ---------- High-level ----------
def correlation_analysis(r: float, n: int):
    if not (-1 <= r <= 1):
        raise ValueError("r must be between -1 and 1")
    if n < 2:
        raise ValueError("n must be ≥ 2")
    dti = compute_dti(r)
    return {"r": r, "n": n, "DTI": dti}

# ---------- CLI ----------
def main():
    print("Correlation Analysis – DTI only (v9.5 compliant)\n")
    r = float(input("Pearson r (-1 to +1): ").strip())
    n = int(input("Sample size n: ").strip())

    res = correlation_analysis(r, n)

    print("\n================ p–fr–nb ================")
    print("p  = n/a for correlations (see header)")
    print("fr = n/a for correlations (see header)")
    print(f"nb (DTI) = {res['DTI']:.6f}")
    print("=========================================\n")

    print("Interpretation:")
    print(generate_narrative(res['r'], res['n'], res['DTI']))

if __name__ == "__main__":
    main()

Correlation Analysis – DTI only (v9.5 compliant)

Pearson r (-1 to +1): 0.5
Sample size n: 32

================ p–fr–nb ================
p  = not part of framework for correlations
fr = not defined (see header)
nb (DTI) = 0.354550

Interpretation:
Pearson r = 0.5000  (n = 32)  (classically “significant” at p≤0.05)
DTI (Distance to Independence) = 0.354550

The observed correlation of r = 0.500 and DTI agree: strong moderately separated from independence.

Remember: DTI is the framework’s official robustness (nb) metric for correlations.
Fragility (fr) is not defined because no unique, model-free way exists to toggle points.
